In [3]:
import numpy as np
import pandas as pd
import corrLib as cl
from skimage import io

I want to do an experiment to see if arbitrary choice of $I_0$ (when computing $\nabla\cdot(cn)$) affects the final result.

In [5]:
def div_field(img, pivData, winsize, step, I0):
    # A function that calculates the divergence field
    # img is the image from microscopy, pivData is a DataFrame with columns (x, y, u, v)
    # winsize and step should be consistent with the parameters of pivData, be extra coutious!
    
    # preprocessing, bpass and match hist for raw image, convert intensity field to density field
    
    # return value: intensity field (subtracted from I0)
    # bp = bpass(img, 3, 100)
    # bp_mh = match_hist(bp, img)
    # winsize = 10
    # step = 10
    X, Y, I = cl.divide_windows(img, windowsize=[winsize, winsize], step=step)
    # concentration field
    c = I0 - I
    
    # calculation for divcn and divcv
    row, col = I.shape
    vx = np.array(pivData.u).reshape(I.shape)
    vy = np.array(pivData.v).reshape(I.shape)
    v = (vx**2 + vy**2)**.5
    nx = np.array(pivData.u / (pivData.u**2 + pivData.v**2)**.5).reshape(I.shape)
    ny = np.array(pivData.v / (pivData.u**2 + pivData.v**2)**.5).reshape(I.shape)
    cnx = c * nx
    cny = c * ny
    cvx = c * vx
    cvy = c * vy
    divcn = np.zeros(I.shape)
    divcv = np.zeros(I.shape)
    divv = np.zeros(I.shape)
    for x in range(0, col-1):
        for y in range(0, row-1):
            divcn[y, x] = cnx[y,x+1] - cnx[y,x] + cny[y+1,x] - cny[y,x]
            divcv[y, x] = cvx[y,x+1] - cvx[y,x] + cvy[y+1,x] - cvy[y,x]
            divv[y, x] = vx[y,x+1] - vx[y,x] + vy[y+1,x] - vy[y,x]
    return c, v, divcn, divcv, divv

In [11]:
img = io.imread(r'E:\moreData\02042020\80-1\8-bit\0000.tif')
pivData = pd.read_csv(r'E:\moreData\02042020\piv_result_50\80-1\0000-0001.csv')
winsize = 50
step = 25
I0 = 500
c, v, divcn1, divcv1, divv1 = div_field(img, pivData, winsize, step, I0)

In [10]:
img = io.imread(r'E:\moreData\02042020\80-1\8-bit\0000.tif')
pivData = pd.read_csv(r'E:\moreData\02042020\piv_result_50\80-1\0000-0001.csv')
winsize = 50
step = 25
I0 = 600
c, v, divcn2, divcv2, divv2 = div_field(img, pivData, winsize, step, I0)

In [12]:
divcn1

array([[-6.22229065e+02, -1.45327647e+02,  2.06224227e+02, ...,
         8.00722506e-02, -2.86687238e+01,  0.00000000e+00],
       [-7.39404214e+02, -3.99909276e+02, -1.38188750e+02, ...,
         2.25259470e+01, -1.76334059e+01,  0.00000000e+00],
       [-2.79156637e+02, -1.97956177e+02,  2.03436649e+02, ...,
         4.57879771e+01,  1.28438623e+02,  0.00000000e+00],
       ...,
       [-7.12750843e-01,  1.14384626e+02,  7.18312669e+01, ...,
         7.54143309e+01,  1.91563977e+02,  0.00000000e+00],
       [-5.35234868e+01,  9.75769463e+01,  1.07077730e+02, ...,
         5.18551051e+01,  1.71655708e+02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [13]:
divcn2

array([[-7.97204056e+02, -1.86539011e+02,  2.64152262e+02, ...,
        -3.71591142e-01, -3.73085405e+01,  0.00000000e+00],
       [-9.47933449e+02, -5.12949724e+02, -1.76986212e+02, ...,
         2.84001120e+01, -2.31929909e+01,  0.00000000e+00],
       [-3.58458740e+02, -2.54216453e+02,  2.61368613e+02, ...,
         5.81095971e+01,  1.64272653e+02,  0.00000000e+00],
       ...,
       [-8.12390110e-01,  1.45934014e+02,  9.18239033e+01, ...,
         9.56651272e+01,  2.43124686e+02,  0.00000000e+00],
       [-6.85143635e+01,  1.24431617e+02,  1.36650818e+02, ...,
         6.51399473e+01,  2.17618847e+02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])